In [12]:
import dask.bag as db
import json
import re
import pandas as pd
import zipfile
import glob
import os

In [13]:
# 1
# Загрузка файлов из архива в виде dask.bag
bag = db.read_text(r'D:\git\3.2_labs\reviews_full\reviews_*.json')

# Преобразование текстового содержимого в объекты Python с помощью модуля json
bag = bag.map(json.loads)

# Вывод первых 5 элементов из полученного bag
print(bag.take(5))

({'user_id': 452355, 'recipe_id': 292657, 'date': '2016-05-08', 'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.'}, {'user_id': 329304, 'recipe_id': 433404, 'date': '2006-06-14', 'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten'}, {'user_id': 227932, 'recipe_id': 2008187, 'date': '1985-11-19', 'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will be making this often.'}, {'user_id': 171468, 'rec

In [14]:
# 2
def parse_json_with_rating(json_str):
    # Компилируем регулярное выражение для извлечения рейтинга из имени файла
    path = re.compile(r'_(\d+)\.json$')
    
    # Преобразуем JSON-строку в объект Python
    data = json.loads(json_str[0])
    
    # Извлекаем рейтинг из имени файла и добавляем его в словарь
    data.update({'rating': int(re.findall(path, json_str[1])[0])})
    
    return data

In [15]:
# 2
# Загрузка файлов из папки в виде dask.bag
bag_2 = db.read_text(r'D:\git\3.2_labs\reviews_full\reviews_*.json', include_path=True)

# Преобразование текстового содержимого в объекты Python с помощью модуля json и добавление рейтинга
bag_21 = bag_2.map(parse_json_with_rating)

""" # для лабы 13
df = bag_21.to_dataframe()
df.to_csv('lab13reviews.csv', index=False) """


# Вывод первых 5 элементов из полученного bag
print(bag_21.take(5))

({'user_id': 452355, 'recipe_id': 292657, 'date': '2016-05-08', 'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.', 'rating': 0}, {'user_id': 329304, 'recipe_id': 433404, 'date': '2006-06-14', 'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten', 'rating': 0}, {'user_id': 227932, 'recipe_id': 2008187, 'date': '1985-11-19', 'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will be making this often.',

In [69]:
# 3
# Загрузка файлов из архива в виде dask.bag
# Подсчет количества отзывов в датасете
count_3 = bag_2.count().compute()

print(f"Количество отзывов: {count_3}")

Количество отзывов: 9057540


In [88]:
# 4
def filter_reviews_by_year(review):
    pattern = re.compile(r'2014|2015')
    if re.findall(pattern, review['date']):
        return review

In [89]:
# 4
bag_4 = bag_21.filter(filter_reviews_by_year)
count_4 = bag_4.count().compute()
print(bag_4.take(5))

({'user_id': 229850, 'recipe_id': 1300038, 'date': '2014-10-03', 'review': 'Took this to a New Year&#039;s Eve Party. Everyone loved it! It&#039;s absolutely perfect, the flavor, the crunch, just delicious!', 'rating': 0}, {'user_id': 2706705, 'recipe_id': 133747, 'date': '2015-05-08', 'review': 'Simple and easy way to enjoy a slice of pizza any time!  Well-toasted bread is the key - really toast it!  I put a bit of pizza sauce underneath my cheese for a more pizza-like flavor.  I used sourdough bread & medium cheddar cheese.  Fast & fun!  Great idea!  Made for 1-2-3 Hits Tag Game.', 'rating': 0}, {'user_id': 945545, 'recipe_id': 898468, 'date': '2015-06-30', 'review': 'Delish!  I wanted to make this spicy so I used hot enchilada sauce and jalapeno refried beans.  I forgot to buy the onions so I doctored up the beans with onion powder and granulated garlic.  Added the olives under the cheese and baked, uncovered, for the 25 minutes.  Served with pico de gallo, sour cream and avocado ch

In [90]:
# 5
def preprocessing(review):
    pattern = re.compile(r'[^a-z\s]')
    review['review'] = re.sub(pattern, '', review['review'].lower().strip())
    return review

In [91]:
# 5
bag_5 = bag_4.map(preprocessing)
print(bag_5.take(5))

({'user_id': 229850, 'recipe_id': 1300038, 'date': '2014-10-03', 'review': 'took this to a new years eve party everyone loved it its absolutely perfect the flavor the crunch just delicious', 'rating': 0}, {'user_id': 2706705, 'recipe_id': 133747, 'date': '2015-05-08', 'review': 'simple and easy way to enjoy a slice of pizza any time  welltoasted bread is the key  really toast it  i put a bit of pizza sauce underneath my cheese for a more pizzalike flavor  i used sourdough bread  medium cheddar cheese  fast  fun  great idea  made for  hits tag game', 'rating': 0}, {'user_id': 945545, 'recipe_id': 898468, 'date': '2015-06-30', 'review': 'delish  i wanted to make this spicy so i used hot enchilada sauce and jalapeno refried beans  i forgot to buy the onions so i doctored up the beans with onion powder and granulated garlic  added the olives under the cheese and baked uncovered for the  minutes  served with pico de gallo sour cream and avocado chunks  fantastic  thanks for sharing lazyme',

In [92]:
# 6
count_6 = bag_5.count().compute()
print(count_6)
if count_6 == count_4:
    print('True')
else:
    print('False')

735274
True


In [93]:
# 7
bag_7 = bag_5.pluck('rating').frequencies().compute()
print(bag_7)

[(0, 42472), (1, 9246), (2, 9380), (3, 26532), (4, 119413), (5, 528231)]


In [94]:
# 8 
bag_8 = bag_5.pluck('rating').mean().compute()
print(bag_8)

4.388036296673077


In [97]:
def max_review_length(total, value):
    return max(total, len(value['review']))


bag_9 = bag_5.foldby(lambda x: x['rating'], max_review_length, 0).compute()
print(bag_9)

[(0, 6548), (1, 2868), (2, 2844), (3, 3174), (4, 6548), (5, 5351)]
